In [2]:
from random import random
from pprint import pprint

import pymongo

In [3]:
c = pymongo.MongoClient()
db = c.db

In [4]:
nb_blocks = 200000

def new_block(branch_ids, block_numbers):
    block = {"branchPath" : {
                            "branchId": branch_ids[0],
                            "blockNumber": block_numbers[0],
                            "branchIds" : branch_ids, 
                            "blockNumbers" : block_numbers},
             "balances": []} 
    for i in [0, 3, 6]:
        if i in branch_ids[1:]:
            continue
        if random() < 0.3:
            block["balances"].append({"address": i, "amount": 10})
    return block

def generate_blocks(n):
    db.drop_collection('blocks')
    db.blocks.create_index([("balances.address", -1), ("branchPath.branchId", -1), ("branchPath.blockNumber", -1)])
    db.blocks.insert_many(new_block([0], [i]) for i in range(n))
    db.blocks.insert_many(new_block([3, 0], [i, n - 1]) for i in range(n))
    db.blocks.insert_many(new_block([6, 3, 0], [i, n - 1]) for i in range(n))

generate_blocks(nb_blocks)

In [9]:
def find_balance(address, branch_path):
    query = {"$or": [
        {
            "balances": {"$elemMatch": {"address": address}}, 
            "branchPath.branchId": branch_id, 
            "branchPath.blockNumber": {"$lte": block_number}
        }
        for branch_id, block_number in zip(branch_path["branchIds"], branch_path["blockNumbers"])
    ]}
    pprint(query)
    return db.blocks.find_one(
        query, 
        sort=[("branchPath.branchId", -1), ("branchPath.blockNumber", -1)]
    )
    
find_balance(3, {"branchIds": [6, 3, 0], "blockNumbers": [623, 19721, nb_blocks - 1]})

{'$or': [{'balances': {'$elemMatch': {'address': 3}},
          'branchPath.blockNumber': {'$lte': 623},
          'branchPath.branchId': 6},
         {'balances': {'$elemMatch': {'address': 3}},
          'branchPath.blockNumber': {'$lte': 19721},
          'branchPath.branchId': 3},
         {'balances': {'$elemMatch': {'address': 3}},
          'branchPath.blockNumber': {'$lte': 199999},
          'branchPath.branchId': 0}]}


{'_id': ObjectId('5d28784e39e3419d0d4b61e3'),
 'branchPath': {'branchId': 3,
  'blockNumber': 19719,
  'branchIds': [3, 0],
  'blockNumbers': [19719, 199999]},
 'balances': [{'address': 3, 'amount': 10}, {'address': 6, 'amount': 10}]}